In [1]:
import sys
import numpy as np
import pandas as pd
import math
import os

In [4]:
#toying with seq num and chrom size to deal with edge cases
sequence_num = 5073
window = 1500
chrom_num=10

#sequence_num = len(file)
seq_length = ((2*int(window))+21)
tot_bases = int(sequence_num*seq_length)
seq_per_chrom = math.floor(sequence_num/chrom_num)
seq_last_chrom = seq_per_chrom + sequence_num%chrom_num
#this is to check if there is the correct amount of sequences per chromosome
scount = seq_last_chrom + (seq_per_chrom)*(chrom_num-1)

base_per_chrom = seq_length*seq_per_chrom 
base_last_chrom = seq_length*seq_last_chrom
#this is to check if there is the correct amount of bases per chromosome
bscount = base_last_chrom + (base_per_chrom)*(chrom_num-1)

print("Individual Sequence Length: " + str(seq_length))
print("Number of Experimental Sequences: " + str(sequence_num)) 
print("Number of Chromosomes: " + str(chrom_num))
print('')
print("Sequences per Chromosome: " + str(seq_per_chrom))
print("Sequences on Last Chromosome: " + str(seq_last_chrom))
if sequence_num-scount != 0:
    print('Error! The total number of sequences does not equal the sum of the sequences divided into chromosomes!')
    sys.exit(1)
else:
    print('Number of sequences per chromosome are properly structured.')
print('')

print("Bases per Chromosome: " + str(base_per_chrom))
print("Bases on Last Chromosome: " + str(base_last_chrom))
print("Total Bases in Experiment: " + str(tot_bases))
if tot_bases-bscount != 0:
    print('Error! The total number of bases does not equal the sum of the bases divided into chromosomes!')
    sys.exit(1)
else:
    print('Number of bases per chromosome are properly structured.')


Individual Sequence Length: 3021
Number of Experimental Sequences: 5073
Number of Chromosomes: 10

Sequences per Chromosome: 507
Sequences on Last Chromosome: 510
Number of sequences per chromosome are properly structured.

Bases per Chromosome: 1531647
Bases on Last Chromosome: 1540710
Total Bases in Experiment: 15325533
Number of bases per chromosome are properly structured.


In [41]:
#funcs needed to run write_fasta
def sequence_input(sequence):
    position_feq = []

    with open(sequence) as seq:

        ##remove the header line
        lines = seq.readlines()[1:]
        for i in range(len(lines)):
            pos_prob = []
            try:
                pos_prob.append(lines[i].strip('\n').split(',')[1])
                pos_prob.append(lines[i].strip('\n').split(',')[2])
                pos_prob.append(lines[i].strip('\n').split(',')[3])
                pos_prob.append(lines[i].strip('\n').split(',')[4])
                position_feq.append(pos_prob)
            except IndexError:
                
                pos_prob.append(lines[i].strip('\n').split('\t')[1])
                pos_prob.append(lines[i].strip('\n').split('\t')[2])
                pos_prob.append(lines[i].strip('\n').split('\t')[3])
                pos_prob.append(lines[i].strip('\n').split('\t')[4])
                position_feq.append(pos_prob)
            else:
                print("Check the input file. \n It should be tab or comma separated")


    return(position_feq)

def sequence_generator(bases,position_feq, sequence_num):
    '''takes in frequencies per position and simulates sequences using a 
    1st order Markov Model.
    '''
    
    first = [position_feq[0:1]]
    last =  [position_feq[-1:]]
    print('Position Frequency: Contains ' + str(len(position_feq)) + " elements. " + str(first) + '...' + str(last))
    
    sequences = np.empty([sequence_num, len(position_feq)], dtype=str)
    
    for i in range(len(position_feq)):
        column = np.random.choice(bases, sequence_num, p=position_feq[i])
        sequences[:,i] = column
        
    joined_sequences = [''.join(row) for row in sequences]
    
    return joined_sequences

In [42]:
#running the scripts to get proper input files for write_fasta
outdir = '/Users/tajo5912/rbg/test'
sample= 'test'

print("-----------Reading Per-Base Sequence Frequency----------------")
tsv = outdir + '/generated_sequences/' + sample + '_base_content.tsv'
print('Base Content File: ' + tsv)
position_prob = sequence_input(tsv)

print("-----------Generating Sequences-------------------------------")
sequences_generating = sequence_generator(bases=['A', 'T', 'G', 'C'], sequence_num=sequence_num, position_feq=position_prob)

# print("-----------Writing Sequences to Fasta File--------------------")
# write_fasta(sequences_generating, sample, outdir, int(window), sequence_num, chrom_num)
# #print('Simulated Fasta File: ' + outdir + '/generated_sequences/' + str(sample) + "_simulated.fa")

-----------Reading Per-Base Sequence Frequency----------------
Base Content File: /Users/tajo5912/rbg/test/generated_sequences/test_base_content.tsv
-----------Generating Sequences-------------------------------
Position Frequency: Contains 3001 elements. [[['0.27503343079259307', '0.26821128260918836', '0.23292196459735726', '0.22383332200086126']]]...[[['0.26763899276988284', '0.2770449445842116', '0.22373699598830488', '0.2315790666576007']]]


In [36]:
#properly formating the output fasta
def write_fasta(generated_sequences, sample, outdir, window, sequence_num, chrom_num):
    ''' writes sequences generated into fasta file
    '''
        
    #checking fasta parameters
    seq_length = ((2*int(window))+21)
    tot_bases = int(sequence_num*seq_length)
    seq_per_chrom = math.floor(sequence_num/chrom_num)
    seq_last_chrom = seq_per_chrom + sequence_num%chrom_num
    #this is to check if there is the correct amount of sequences per chromosome
    scount = seq_last_chrom + (seq_per_chrom)*(chrom_num-1)

    base_per_chrom = seq_length*seq_per_chrom 
    base_last_chrom = seq_length*seq_last_chrom
    #this is to check if there is the correct amount of bases per chromosome
    bscount = base_last_chrom + (base_per_chrom)*(chrom_num-1)

    print("Individual Sequence Length: " + str(seq_length))
    print("Number of Experimental Sequences: " + str(sequence_num)) 
    print("Number of Chromosomes: " + str(chrom_num) +'\n')
    print("Sequences per Chromosome: " + str(seq_per_chrom))
    print("Sequences on Last Chromosome: " + str(seq_last_chrom))
    if sequence_num-scount != 0:
        print('Error! The total number of sequences does not equal the sum of the sequences divided into chromosomes!')
        sys.exit(1)
    else:
        print('Number of sequences per chromosome are properly structured.\n')

    print("Bases per Chromosome: " + str(base_per_chrom))
    print("Bases on Last Chromosome: " + str(base_last_chrom))
    print("Total Bases in Experiment: " + str(tot_bases))
    if tot_bases-bscount != 0:
        print('Error! The total number of bases does not equal the sum of the bases divided into chromosomes!')
        sys.exit(1)
    else:
        print('Number of bases per chromosome are properly structured.')
    
    file=[]
    for i in range(len(generated_sequences)):
        file.append(str(generated_sequences[i]) + str('N')*20)      

    dd = {}
    for i in range(0,chrom_num-1):
        dd['>sim_' + str(i + 1)] = str(file[(seq_per_chrom)*i:(seq_per_chrom)*(i+1)]).replace("', '","").replace("['","").replace("']","")
    dd['>sim_' + str(chrom_num)] = str(file[(seq_per_chrom)*(chrom_num-1):]).replace("', '","").replace("['","").replace("']","")

    df = pd.DataFrame.from_dict(dd, orient='index')

    df = df.reset_index()
    df.columns = range(df.shape[1])
    df = df.stack()
    df = pd.DataFrame(df).reset_index(drop=True)
    return df.to_csv(outdir + "/generated_sequences/" + str(sample) + "_simulated.fa", header=None, index=False, sep='\t')

In [87]:
#accounting for the last chromosome where all chromosomes are the same length and the last chromosome picks up the remaining sequences
generated_sequences= sequences_generating
file=[]
for i in range(len(generated_sequences)):
    file.append(str(generated_sequences[i]) + str('N')*20)      

dd = {}
for i in range(0,chrom_num-1):
    dd['>sim_' + str(i + 1)] = str(file[(seq_per_chrom)*i:(seq_per_chrom)*(i+1)]).replace("', '","").replace("['","").replace("']","")
dd['>sim_' + str(chrom_num)] = str(file[(seq_per_chrom)*(chrom_num-1):]).replace("', '","").replace("['","").replace("']","")
    
df = pd.DataFrame.from_dict(dd, orient='index')

df = df.reset_index()
df.columns = range(df.shape[1])
df = df.stack()
df = pd.DataFrame(df).reset_index(drop=True)

df
# df.columns = ['seq']
# def base_per_chromosome(row):
#     r = pd.Series(row['seq'])
#     x = len(row['seq'])
#     return x

# df['counts'] = df.apply(lambda row: base_per_chromosome(row), axis=1)
# # df #1531647 on chrm 1-9
# # #what about chrm 10
# df

,seq,counts
0,>sim_1,6
1,ATCTAGACCGTAAGCGATCATGCAACTACATTGAGGGGGAATCTAG...,1531647
2,>sim_2,6
3,GGGGTAGGTTCCTAAATGTATTCATTTTGCAGAGGGCCTAGTTGCA...,1531647
4,>sim_3,6
5,GCTTAGGGGTCAAAGACGTCGCTAGAATTATATCCGGTTCCCAAGT...,1531647
6,>sim_4,6
7,GTGCGTCTAATCAAGCCTGAAGGTAATGGGACCCTGAATTTAAGAG...,1531647
8,>sim_5,6
9,TGGGATTTTTCAGAATCGTACTATAATCGTACTTGGTCAGCTCCGT...,1531647


In [82]:
listf = [1,2,3,4,5,6,7,8,9,10]
listf[4:]
last_chrom = str(file[(seq_per_chrom)*(chrom_num-1):]).replace("', '","").replace("['","").replace("']","")
len(last_chrom)

1540710

In [92]:
len(generated_sequences)
sequence_num = 5073
window = 1500
chrom_num=10

#sequence_num = len(file)
seq_length = ((2*int(window))+21)
tot_bases = int(sequence_num*seq_length)
seq_per_chrom = math.floor(sequence_num/chrom_num)
seq_last_chrom = seq_per_chrom + sequence_num%chrom_num
#this is to check if there is the correct amount of sequences per chromosome
scount = seq_last_chrom + (seq_per_chrom)*(chrom_num-1)

base_per_chrom = seq_length*seq_per_chrom 
base_last_chrom = seq_length*seq_last_chrom
#this is to check if there is the correct amount of bases per chromosome
bscount = base_last_chrom + (base_per_chrom)*(chrom_num-1)

print("Individual Sequence Length: " + str(seq_length))
print("Number of Experimental Sequences: " + str(sequence_num)) 
print("Number of Chromosomes: " + str(chrom_num))
print('')
print("Sequences per Chromosome: " + str(seq_per_chrom))
print("Sequences on Last Chromosome: " + str(seq_last_chrom))
if sequence_num-scount != 0:
    print('Error! The total number of sequences does not equal the sum of the sequences divided into chromosomes!')
    sys.exit(1)
else:
    print('Number of sequences per chromosome are properly structured.')
print('')

print("Bases per Chromosome: " + str(base_per_chrom))
print("Bases on Last Chromosome: " + str(base_last_chrom))
print("Total Bases in Experiment: " + str(tot_bases))
if tot_bases-bscount != 0:
    print('Error! The total number of bases does not equal the sum of the bases divided into chromosomes!')
    sys.exit(1)
else:
    print('Number of bases per chromosome are properly structured.')


Individual Sequence Length: 3021
Number of Experimental Sequences: 5073
Number of Chromosomes: 10

Sequences per Chromosome: 507
Sequences on Last Chromosome: 510
Number of sequences per chromosome are properly structured.

Bases per Chromosome: 1531647
Bases on Last Chromosome: 1540710
Total Bases in Experiment: 15325533
Number of bases per chromosome are properly structured.


In [117]:
#for inserting in the python script
def sim_bed(self):
    #calculating the number of sequences per chromosome
    seq_per_chrom = math.floor(self.sequence_num/self.chrom_num)
    seq_last_chrom = seq_per_chrom + self.sequence_num%self.chrom_num
    #creating the locations for the all chromosomes except the last
    dbed = {}
    for i in range(0,(seq_per_chrom)):
        dbed['r_' + str(i)] = [int(self.window-1)+i*(2*self.window+21), int(self.window+1)+i*(2*self.window+21)]

    dfbed = pd.DataFrame.from_dict(dbed, orient='index')
    dfbed.columns = ['start', 'stop']
    #duplicating the dataframe for all chromosomes except the last
    dfbed = pd.concat([dfbed]*(self.chrom_num-1))
    #adding the chromosome name to the genomic location
    l_sim=[]
    for i in range(int(seq_per_chrom)):
        for c in range(int(self.chrom_num-1)):
            l_sim.append(str('sim_' + str(c+1)))
    l_sim = sorted(l_sim)
    dfbed['sim'] = l_sim

    #creating the locations for the last chromosome
    dlast = {}
    for i in range(0,(seq_last_chrom)):
        dlast['l_' + str(i)] = [int(self.window-1)+i*(2*self.window+21), int(self.window+1)+i*(2*self.window+21)]
    dflast = pd.DataFrame.from_dict(dlast, orient='index')
    dflast.columns = ['start', 'stop']
    #adding the chromosome name to the genomic location
    l_last = []
    for i in range(int(seq_last_chrom)):
        l_last.append(str('sim_' + str(self.chrom_num)))
    dflast['sim'] = l_last

    #combining the bed locations
    df = pd.concat([dfbed, dflast])
    df = df[['sim', 'start', 'stop']]
    #saving the new annotation
    df.to_csv(self.outdir + "/annotations/" + str(self.sample) + "_simulated_centered.bed", header=None, index=False, sep='\t')

    #creating the windowed bed
    df["start_new"] = df.apply(lambda x: round((x["start"] + x["stop"])/2), axis=1)
    df["stop_new"] = df.apply(lambda x: x["start_new"] + 1, axis = 1)

    ##the -1500 position from "origin"
    df["start"] = df.apply(lambda x: x["start_new"] - int(self.window), axis=1)

    ##the 1500 position from the "origin"
    df["stop"] = df.apply(lambda x: x["stop_new"] + int(self.window), axis=1)

    ##saving the new annotation
    df.to_csv(self.outdir + '/annotations/' + self.sample + '_simulated_window.bed', sep='\t', columns=["sim","start","stop"], header = False, index = False)
    return print('Simulated Bed Files Genereated: ' + self.outdir + "/annotations/" + str(self.sample) + "_simulated_centered.bed and " + str(self.sample) + "_simulated_self.window.bed")

In [20]:
##for testing in jup notebook
def sim_bed(sequence_num, chrom_num, window, outdir, sample):
    #calculating the number of sequences per chromosome
    seq_per_chrom = math.floor(sequence_num/chrom_num)
    seq_last_chrom = seq_per_chrom + sequence_num%chrom_num
    scount = seq_last_chrom + (seq_per_chrom)*(chrom_num-1)
    #creating the locations for the all chromosomes except the last
    
    print("----------Checking bed parameters------------")
    print("Individual Sequence Length: " + str(seq_length))
    print("Number of Experimental Sequences: " + str(sequence_num)) 
    print("Number of Chromosomes: " + str(chrom_num) +'\n')
    print("Sequences per Chromosome: " + str(seq_per_chrom))
    print("Sequences on Last Chromosome: " + str(seq_last_chrom))
    if sequence_num-scount != 0:
        print('Error! The total number of sequences does not equal the sum of the sequences divided into chromosomes!')
        sys.exit(1)
    else:
        print('Number of sequences per chromosome are properly structured.\n')

    dbed = {}
    for i in range(0,(seq_per_chrom)):
        dbed['r_' + str(i)] = [int(window-1)+i*(2*window+21), int(window+1)+i*(2*window+21)]

    dfbed = pd.DataFrame.from_dict(dbed, orient='index')
    dfbed.columns = ['start', 'stop']
    #duplicating the dataframe for all chromosomes except the last
    dfbed = pd.concat([dfbed]*(chrom_num-1))
    #adding the chromosome name to the genomic location
    l_sim=[]
    for i in range(int(seq_per_chrom)):
        for c in range(int(chrom_num-1)):
            l_sim.append(str('sim_' + str(c+1)))
    l_sim = sorted(l_sim)
    dfbed['sim'] = l_sim

    #creating the locations for the last chromosome
    dlast = {}
    for i in range(0,(seq_last_chrom)):
        dlast['l_' + str(i)] = [int(window-1)+i*(2*window+21), int(window+1)+i*(2*window+21)]
    dflast = pd.DataFrame.from_dict(dlast, orient='index')
    dflast.columns = ['start', 'stop']
    #adding the chromosome name to the genomic location
    l_last = []
    for i in range(int(seq_last_chrom)):
        l_last.append(str('sim_' + str(chrom_num)))
    dflast['sim'] = l_last

    #combining the bed locations
    df = pd.concat([dfbed, dflast])
    df = df[['sim', 'start', 'stop']]
    #saving the new annotation
    df.to_csv(outdir + "/annotations/" + str(sample) + "_simulated_centered.bed", header=None, index=False, sep='\t')

    #creating the windowed bed
    df["start_new"] = df.apply(lambda x: round((x["start"] + x["stop"])/2), axis=1)
    df["stop_new"] = df.apply(lambda x: x["start_new"] + 1, axis = 1)

    ##the -1500 position from "origin"
    df["start"] = df.apply(lambda x: x["start_new"] - int(window), axis=1)

    ##the 1500 position from the "origin"
    df["stop"] = df.apply(lambda x: x["stop_new"] + int(window), axis=1)

    ##saving the new annotation
    df.to_csv(outdir + '/annotations/' + sample + '_simulated_window.bed', sep='\t', columns=["sim","start","stop"], header = False, index = False)
    return print('Simulated Bed Files Genereated: ' + outdir + "/annotations/" + str(sample) + "_simulated_centered.bed and " + str(sample) + "_simulated_window.bed")

sim_bed(sequence_num=5073, chrom_num=10, window=1500, outdir='/Users/tajo5912/rbg/test', sample='test')

----------Checking bed parameters------------
Individual Sequence Length: 3021
Number of Experimental Sequences: 5073
Number of Chromosomes: 10

Sequences per Chromosome: 507
Sequences on Last Chromosome: 510
Number of sequences per chromosome are properly structured.

Simulated Bed Files Genereated: /Users/tajo5912/rbg/test/annotations/test_simulated_centered.bed and test_simulated_window.bed


Now to reformat the experimental "genome" ie the sequences based on the annotation file to generate its own genome

In [12]:
def reformat_expt_fasta(outdir, sample, window, chrom_num):
    df = pd.read_csv(outdir + "/generated_sequences/" + sample + "_experimental_window_sequences.fa", header=None)
    df = df[df.index % 2 != 0]
    l_expt_seq = df.values.tolist()
    flat_list = [item for sublist in l_expt_seq for item in sublist]
    strN = str('N')*20
    file = ["{}{}".format(i,strN) for i in flat_list]

    sequence_num = len(file)
    seq_length = ((2*int(window))+21)
    tot_bases = int(sequence_num*seq_length)
    seq_per_chrom = math.floor(sequence_num/chrom_num)
    seq_last_chrom = seq_per_chrom + sequence_num%chrom_num
    #this is to check if there is the correct amount of sequences per chromosome
    scount = seq_last_chrom + (seq_per_chrom)*(chrom_num-1)

    base_per_chrom = seq_length*seq_per_chrom 
    base_last_chrom = seq_length*seq_last_chrom
    #this is to check if there is the correct amount of bases per chromosome
    bscount = base_last_chrom + (base_per_chrom)*(chrom_num-1)

    print("Individual Sequence Length: " + str(seq_length))
    print("Number of Experimental Sequences: " + str(sequence_num)) 
    print("Number of Chromosomes: " + str(chrom_num) +'\n')
    print("Sequences per Chromosome: " + str(seq_per_chrom))
    print("Sequences on Last Chromosome: " + str(seq_last_chrom))
    if sequence_num-scount != 0:
        print('Error! The total number of sequences does not equal the sum of the sequences divided into chromosomes!')
        sys.exit(1)
    else:
        print('Number of sequences per chromosome are properly structured.\n')

    print("Bases per Chromosome: " + str(base_per_chrom))
    print("Bases on Last Chromosome: " + str(base_last_chrom))
    print("Total Bases in Experiment: " + str(tot_bases))
    if tot_bases-bscount != 0:
        print('Error! The total number of bases does not equal the sum of the bases divided into chromosomes!')
        sys.exit(1)
    else:
        print('Number of bases per chromosome are properly structured.')

    dd = {}
    for i in range(0,chrom_num-1):
        dd['>exp_' + str(i + 1)] = str(file[(seq_per_chrom)*i:(seq_per_chrom)*(i+1)]).replace("', '","").replace("['","").replace("']","")
    dd['>exp_' + str(chrom_num)] = str(file[(seq_per_chrom)*(chrom_num-1):]).replace("', '","").replace("['","").replace("']","")
    df = pd.DataFrame.from_dict(dd, orient='index')
    df.columns = range(df.shape[1])
    df=df.reset_index()
    df = df.stack()
    df = pd.DataFrame(df).reset_index(drop=True)
    return df.to_csv(outdir + "/generated_sequences/" + str(sample) + "_experimental.fa", header=None, index=False, sep='\t')
reformat_expt_fasta('/Users/tajo5912/rbg/test', 'test', 1500,10)

Individual Sequence Length: 3021
Number of Experimental Sequences: 44121
Number of Chromosomes: 10

Sequences per Chromosome: 4412
Sequences on Last Chromosome: 4413
Number of sequences per chromosome are properly structured.

Bases per Chromosome: 13328652
Bases on Last Chromosome: 13331673
Total Bases in Experiment: 133289541
Number of bases per chromosome are properly structured.


In [ ]:
def reformat_expt_fasta(self):
    df = pd.read_csv(self.outdir + "/generated_sequences/" + self.sample + "_experimental_self.window_sequences.fa", header=None)
    df = df[df.index % 2 != 0]
    l_expt_seq = df.values.tolist()
    flat_list = [item for sublist in l_expt_seq for item in sublist]
    strN = str('N')*20
    file = ["{}{}".format(i,strN) for i in flat_list]

    sequence_num = len(file)
    seq_length = ((2*int(self.window))+21)
    tot_bases = int(sequence_num*seq_length)
    seq_per_chrom = math.floor(sequence_num/self.chrom_num)
    seq_last_chrom = seq_per_chrom + sequence_num%self.chrom_num
    #this is to check if there is the correct amount of sequences per chromosome
    scount = seq_last_chrom + (seq_per_chrom)*(self.chrom_num-1)

    base_per_chrom = seq_length*seq_per_chrom 
    base_last_chrom = seq_length*seq_last_chrom
    #this is to check if there is the correct amount of bases per chromosome
    bscount = base_last_chrom + (base_per_chrom)*(self.chrom_num-1)

    print("Individual Sequence Length: " + str(seq_length))
    print("Number of Experimental Sequences: " + str(sequence_num)) 
    print("Number of Chromosomes: " + str(self.chrom_num) +'\n')
    print("Sequences per Chromosome: " + str(seq_per_chrom))
    print("Sequences on Last Chromosome: " + str(seq_last_chrom))
    if sequence_num-scount != 0:
        print('Error! The total number of sequences does not equal the sum of the sequences divided into chromosomes!')
        sys.exit(1)
    else:
        print('Number of sequences per chromosome are properly structured.\n')

    print("Bases per Chromosome: " + str(base_per_chrom))
    print("Bases on Last Chromosome: " + str(base_last_chrom))
    print("Total Bases in Experiment: " + str(tot_bases))
    if tot_bases-bscount != 0:
        print('Error! The total number of bases does not equal the sum of the bases divided into chromosomes!')
        sys.exit(1)
    else:
        print('Number of bases per chromosome are properly structured.')

    dd = {}
    for i in range(0,self.chrom_num-1):
        dd['>exp_' + str(i + 1)] = str(file[(seq_per_chrom)*i:(seq_per_chrom)*(i+1)]).replace("', '","").replace("['","").replace("']","")
    dd['>exp_' + str(self.chrom_num)] = str(file[(seq_per_chrom)*(self.chrom_num-1):]).replace("', '","").replace("['","").replace("']","")
    df = pd.DataFrame.from_dict(dd, orient='index')
    df.columns = range(df.shape[1])
    df=df.reset_index()
    df = df.stack()
    df = pd.DataFrame(df).reset_index(drop=True)
    return df.to_csv(self.outdir + "/generated_sequences/" + str(self.sample) + "_experimental.fa", header=None, index=False, sep='\t')

In [5]:
##for testing in jup notebook
def expt_bed(annotation, chrom_num, window, outdir, sample):
    annotation_file = pd.read_csv(annotation, header=None)
    sequence_num = len(annotation_file)
    
    #calculating the number of sequences per chromosome
    seq_per_chrom = math.floor(sequence_num/chrom_num)
    seq_last_chrom = seq_per_chrom + sequence_num%chrom_num
    seq_length = ((2*int(window))+21)
    tot_bases = int(sequence_num*seq_length)
    #this is to check if there is the correct amount of sequences per chromosome
    scount = seq_last_chrom + (seq_per_chrom)*(chrom_num-1)
    
    print("----------Checking bed parameters------------")
    print("Individual Sequence Length: " + str(seq_length))
    print("Number of Experimental Sequences: " + str(sequence_num)) 
    print("Number of Chromosomes: " + str(chrom_num) +'\n')
    print("Sequences per Chromosome: " + str(seq_per_chrom))
    print("Sequences on Last Chromosome: " + str(seq_last_chrom))
    if sequence_num-scount != 0:
        print('Error! The total number of sequences does not equal the sum of the sequences divided into chromosomes!')
        sys.exit(1)
    else:
        print('Number of sequences per chromosome are properly structured.\n')
    
    
    #creating the locations for the all chromosomes except the last
    dbed = {}
    for i in range(0,(seq_per_chrom)):
        dbed['r_' + str(i)] = [int(window-1)+i*(2*window+21), int(window+1)+i*(2*window+21)]

    dfbed = pd.DataFrame.from_dict(dbed, orient='index')
    dfbed.columns = ['start', 'stop']
    #duplicating the dataframe for all chromosomes except the last
    dfbed = pd.concat([dfbed]*(chrom_num-1))
    #adding the chromosome name to the genomic location
    l_expt=[]
    for i in range(int(seq_per_chrom)):
        for c in range(int(chrom_num-1)):
            l_expt.append(str('exp_' + str(c+1)))
    l_expt = sorted(l_expt)
    dfbed['expt'] = l_expt

    #creating the locations for the last chromosome
    dlast = {}
    for i in range(0,(seq_last_chrom)):
        dlast['l_' + str(i)] = [int(window-1)+i*(2*window+21), int(window+1)+i*(2*window+21)]
    dflast = pd.DataFrame.from_dict(dlast, orient='index')
    dflast.columns = ['start', 'stop']
    #adding the chromosome name to the genomic location
    l_last = []
    for i in range(int(seq_last_chrom)):
        l_last.append(str('exp_' + str(chrom_num)))
    dflast['expt'] = l_last

    #combining the bed locations
    df = pd.concat([dfbed, dflast])
    df = df[['expt', 'start', 'stop']]
    #saving the new annotation
    df.to_csv(outdir + "/annotations/" + str(sample) + "_experimental_genome_centered.bed", header=None, index=False, sep='\t')

    #creating the windowed bed
    df["start_new"] = df.apply(lambda x: round((x["start"] + x["stop"])/2), axis=1)
    df["stop_new"] = df.apply(lambda x: x["start_new"] + 1, axis = 1)

    ##the -1500 position from "origin"
    df["start"] = df.apply(lambda x: x["start_new"] - int(window), axis=1)

    ##the 1500 position from the "origin"
    df["stop"] = df.apply(lambda x: x["stop_new"] + int(window), axis=1)
    
    ##saving the new annotation
    df.to_csv(outdir + '/annotations/' + sample + '_experimental_genome_window.bed', sep='\t', columns=["expt","start","stop"], header = False, index = False)    
    return print('Experimental Bed Files Genereated: ' + outdir + "/annotations/" + str(sample) + "_experimental_genome_centered.bed and " + str(sample) + "_experimental_genome_window.bed")

expt_bed(annotation='/scratch/Users/tajo5912/rbg/levandow_mumerge_bidir.bed', chrom_num=10, window=1500, outdir='/scratch/Users/tajo5912/rbg/levandowski2021', sample='levandowski2021')



----------Checking bed parameters------------
Individual Sequence Length: 3021
Number of Experimental Sequences: 44121
Number of Chromosomes: 10

Sequences per Chromosome: 4412
Sequences on Last Chromosome: 4413
Number of sequences per chromosome are properly structured.

Experimental Bed Files Genereated: /scratch/Users/tajo5912/rbg/levandowski2021/annotations/levandowski2021_experimental_genome_centered.bed and levandowski2021_experimental_genome_window.bed


In [9]:
outdir = '/Users/tajo5912/rbg/test'
annotation = '/scratch/Users/tajo5912/rbg/levandow_mumerge_bidir.bed'
sample='test'
df = pd.read_csv(annotation, header=None)
len(df) 

44121

In [ ]:
##for main
def expt_bed(self):
    annotation_file = pd.read_csv(self.annotation, header=None)
    sequence_num = len(annotation_file)
    
    #calculating the number of sequences per chromosome
    seq_per_chrom = math.floor(sequence_num/self.chrom_num)
    seq_last_chrom = seq_per_chrom + sequence_num%self.chrom_num
    scount = seq_last_chrom + (seq_per_chrom)*(self.chrom_num-1)
    
    print("----------Checking bed parameters------------")
    print("Individual Sequence Length: " + str(seq_length))
    print("Number of Experimental Sequences: " + str(sequence_num)) 
    print("Number of Chromosomes: " + str(self.chrom_num) +'\n')
    print("Sequences per Chromosome: " + str(seq_per_chrom))
    print("Sequences on Last Chromosome: " + str(seq_last_chrom))
    if sequence_num-scount != 0:
        print('Error! The total number of sequences does not equal the sum of the sequences divided into chromosomes!')
        sys.exit(1)
    else:
        print('Number of sequences per chromosome are properly structured.\n')
    
    
    #creating the locations for the all chromosomes except the last
    dbed = {}
    for i in range(0,(seq_per_chrom)):
        dbed['r_' + str(i)] = [int(self.window-1)+i*(2*self.window+21), int(self.window+1)+i*(2*self.window+21)]

    dfbed = pd.DataFrame.from_dict(dbed, orient='index')
    dfbed.columns = ['start', 'stop']
    #duplicating the dataframe for all chromosomes except the last
    dfbed = pd.concat([dfbed]*(self.chrom_num-1))
    #adding the chromosome name to the genomic location
    l_expt=[]
    for i in range(int(seq_per_chrom)):
        for c in range(int(self.chrom_num-1)):
            l_expt.append(str('expt_' + str(c+1)))
    l_expt = sorted(l_expt)
    dfbed['expt'] = l_expt

    #creating the locations for the last chromosome
    dlast = {}
    for i in range(0,(seq_last_chrom)):
        dlast['l_' + str(i)] = [int(self.window-1)+i*(2*self.window+21), int(self.window+1)+i*(2*self.window+21)]
    dflast = pd.DataFrame.from_dict(dlast, orient='index')
    dflast.columns = ['start', 'stop']
    #adding the chromosome name to the genomic location
    l_last = []
    for i in range(int(seq_last_chrom)):
        l_last.append(str('expt_' + str(self.chrom_num)))
    dflast['expt'] = l_last

    #combining the bed locations
    df = pd.concat([dfbed, dflast])
    df = df[['expt', 'start', 'stop']]
    #saving the new annotation
    df.to_csv(self.outdir + "/annotations/" + str(self.sample) + "_experimental_genome_centered.bed", header=None, index=False, sep='\t')

    #creating the self.windowed bed
    df["start_new"] = df.apply(lambda x: round((x["start"] + x["stop"])/2), axis=1)
    df["stop_new"] = df.apply(lambda x: x["start_new"] + 1, axis = 1)

    ##the -1500 position from "origin"
    df["start"] = df.apply(lambda x: x["start_new"] - int(self.window), axis=1)

    ##the 1500 position from the "origin"
    df["stop"] = df.apply(lambda x: x["stop_new"] + int(self.window), axis=1)
    
    ##saving the new annotation
    df.to_csv(self.outdir + '/annotations/' + self.sample + '_experimental_genome_self.window.bed', sep='\t', columns=["expt","start","stop"], header = False, index = False)    
    return print('Experimental Bed Files Genereated: ' + self.outdir + "/annotations/" + str(self.sample) + "_experimental_genome_centered.bed and " + str(self.sample) + "_experimental_genome_self.window.bed")


In [3]:
df = pd.read_csv('/scratch/Users/tajo5912/rbg/levandowski2021/annotations/levandowski2021_simulated_centered.bed', header= None, sep='\t')


df=df.iloc[0:5001]
df.to_csv('/Users/tajo5912/rbg/md_testing/5000_sim.bed', header=None, index=False, sep='\t')

